In [ ]:
#!pip install requests_oauthlib
import socket
import sys
import requests
import requests_oauthlib
import json
import os


# Replace the values below with yours
ACCESS_TOKEN = '1393813612460789763-mAC5fJM9wb0pQgQqKj5k8yGXxiSWxd'
ACCESS_SECRET = 'ww6kZDUTSMy2kqR04ywfgZPmej1fG73iQEU5sfQn2ssZf'
CONSUMER_KEY = 'ehiVqgsQhfvvfQKSAJQh4tfIR'
CONSUMER_SECRET = 'IJ7m5RTDtFyHkGriOtgQLDttSvmCuI39bOq3dPkqHs1WPU7qOh'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

bearer_token = "AAAAAAAAAAAAAAAAAAAAAANZZwEAAAAAPrqDwIuYK6glR%2FQ74GrGsCFcilw%3DgjyGwQY0Tn0tZBZ31deJqUF7oz1MjlKoXwk342o9X5KkQyTp6c"

# global count
# count = 0

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "crypto", "tag": "crypto"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    count = 0
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    return response
    # for response_line in response.iter_lines():
    #     if response_line:
    #         json_response = json.loads(response_line)
    #         print(json.dumps(json_response, indent=4, sort_keys=True))
    #         count += 1
    #         if count > 100: 
    #             count = 0
    #             break
    # return json_response
            


def get_tweet_stream():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    print("get_tweet_stream function runs successfully")
    return get_stream(set)


def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            tweet = json.loads(line)['data']['text']
            print("Tweet Text: " + tweet)
            print ("------------------------------------------")
            tcp_connection.sendall((tweet + '\n').encode('utf-8'))
            #tcp_connection.send(json.dumps(tweet_text).encode('utf-8'))
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)



if __name__ == "__main__":
  s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)         # Create a socket object
  TCP_IP = "localhost"
  TCP_PORT = 10711              # Reserve a port for your service.
  s.bind((TCP_IP, TCP_PORT))        # Bind to the port

  print("Listening on port: %s" % str(TCP_PORT))
  s.listen(5)                 # Now wait for client connection.
  tcp_connection, addr = s.accept()        # Establish connection with client.
  print( "Received request from: " + str( addr ) )

  resp = get_tweet_stream() #here
  print("This is the http response")
  send_tweets_to_spark(resp, tcp_connection)

Listening on port: 10711
Received request from: ('127.0.0.1', 40236)
{"data": [{"id": "1519739532861743107", "value": "crypto", "tag": "crypto"}], "meta": {"sent": "2022-04-28T18:25:04.838Z", "result_count": 1}}
{"meta": {"sent": "2022-04-28T18:25:05.705Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "crypto", "tag": "crypto", "id": "1519744528953004036"}], "meta": {"sent": "2022-04-28T18:25:07.201Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
get_tweet_stream function runs successfully
200
This is the http response
Tweet Text: RT @PetsCrossing: If you are seeing this, you are early. 

Pets Crossing is a metaverse game currently on alpha stage of development. 

RT…
------------------------------------------
Tweet Text: Well! This is pretty big news! #XRP #ETH #Ripple #xrpthestandard #Crypto #NFTGiveaway

https://t.co/v6CXICINiU
------------------------------------------
Tweet Text: RT @J_Ladiva: Is today the day team United reclaim the